In [1]:
!conda info


     active environment : base
    active env location : C:\Users\Nene\anaconda3
            shell level : 1
       user config file : C:\Users\Nene\.condarc
 populated config files : C:\Users\Nene\.condarc
          conda version : 4.8.3
    conda-build version : 3.18.11
         python version : 3.8.3.final.0
       virtual packages : 
       base environment : C:\Users\Nene\anaconda3  (writable)
           channel URLs : https://repo.anaconda.com/pkgs/main/win-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/win-64
                          https://repo.anaconda.com/pkgs/r/noarch
                          https://repo.anaconda.com/pkgs/msys2/win-64
                          https://repo.anaconda.com/pkgs/msys2/noarch
          package cache : C:\Users\Nene\anaconda3\pkgs
                          C:\Users\Nene\.conda\pkgs
                          C:\Users\Nene\AppData\Local\conda\conda\pkgs
       en

In [42]:
from math import pi

import pandas as pd

from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum

output_notebook() 

x = {
    'United States': 157,
    'United Kingdom': 93,
    'Japan': 89,
    'China': 63,
    'Germany': 44,
    'India': 42,
    'Italy': 40,
    'Australia': 35,
    'Brazil': 32,
    'France': 31,
    'Taiwan': 31,
    'Spain': 29
}
numRace = len(x)
print(numRace)
data = pd.Series(x)
print(data.head())

data = data.reset_index(name='value').rename(columns={'index':'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[numRace]

print(data.head())

p = figure(plot_height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='country', source=data)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

#show(p)

Loading BokehJS ...

12
United States     157
United Kingdom     93
Japan              89
China              63
Germany            44
dtype: int64
          country  value     angle    color
0   United States    157  1.437988  #3182bd
1  United Kingdom     93  0.851802  #6baed6
2           Japan     89  0.815165  #9ecae1
3           China     63  0.577027  #c6dbef
4         Germany     44  0.403003  #e6550d


In [15]:
import pandas as pd
import datetime
import numpy as np
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource, curdoc
from bokeh.io import push_notebook
from bokeh.models import HoverTool
from ipywidgets import interact
from bokeh.palettes import Category20c

In [92]:
dataRace = pd.read_csv('../cdph-race-ethnicity.csv')
dataRace['date_time'] = pd.to_datetime(dataRace['date'])
dataRace = dataRace.loc[dataRace['age'] == 'all']
dataRace = dataRace.sort_values(by=['date_time', 'race'], ascending=True)
dataRace = dataRace[['date_time', 'race', 'confirmed_cases_percent', 'deaths_percent']]
numRace = len(dataRace['race'].unique())

dataRaceTotal = dataRace.groupby(['date_time']).agg({'confirmed_cases_percent': 'sum', 'deaths_percent': 'sum'}).rename(columns={'confirmed_cases_percent':'total_cases_percent', 'deaths_percent': 'total_deaths_percent'})
dataRace = pd.merge(dataRace, dataRaceTotal, on='date_time')
#dataRace.head()

dataRaceSelected = dataRace.loc[dataRace['date_time'] == "2020-05-15"].copy()
dataRaceSelected.reset_index()#.rename(columns={'index':'raceIdx'})
dataRaceSelected['angle_cases'] = dataRaceSelected['confirmed_cases_percent']/dataRaceSelected['total_cases_percent'] * 2*pi
dataRaceSelected['angle_deaths'] = dataRaceSelected['deaths_percent']/dataRaceSelected['total_deaths_percent'] * 2*pi
dataRaceSelected['color'] = Category20c[numRace]
dataRaceSelected.head()

#dataRace.groupby(['date', 'race']).agg({'confirmed_cases_total': ['min', 'max'], 'population_percent': 'sum'}).head(30)
#df.loc[(df['column_name'] >= A) & (df['column_name'] <= B)]
#dataRace['angle'] = dataRace['confirmed_cases_percent']/dataRaceTotal['value'].sum() * 2*pi
#dataRace['color'] = Category20c[numRace]
#dataRace.shape

pieDf=pd.DataFrame({'race':dataRaceSelected['race'], 'percentage':dataRaceSelected['confirmed_cases_percent'], 'angle':dataRaceSelected['angle_cases'], 'color': dataRaceSelected['color']})
pieSource = ColumnDataSource(pieDf)

output_notebook() 
pieFig = figure(plot_height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips=("Race @race: @percentage{:.0%}"))

piePlot = pieFig.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='race', source=pieSource)

pieFig.axis.axis_label=None
pieFig.axis.visible=False
pieFig.grid.grid_line_color = None

Loading BokehJS ...

In [93]:
def updatePie(sel):
    if sel=='Cases':
        piePlot.data_source.data['percentage']=dataRaceSelected['confirmed_cases_percent']
        piePlot.data_source.data['angle']=dataRaceSelected['angle_cases']
    elif sel == 'Deaths':
        piePlot.data_source.data['percentage']=dataRaceSelected['deaths_percent']
        piePlot.data_source.data['angle']=dataRaceSelected['angle_deaths']
    push_notebook()

show(pieFig, notebook_handle=True)

In [94]:
updatePie("Deaths")

In [16]:
data = pd.read_csv('../cdph-state-totals.csv')

data['date_time'] = pd.to_datetime(data['date'])
data = data.sort_values(by=['date_time'], ascending=True)
data['new_confirmed_cases'] = data['confirmed_cases'].diff().fillna(data['confirmed_cases'])
data['new_deaths'] = data['deaths'].diff().fillna(0)
data.head(50)

,date,confirmed_cases,deaths,travel,person_to_person,community_spread,under_investigation,other_causes,self_monitoring,age_0_to_17,...,confirmed_hospitalizations,confirmed_icu,suspected_hospitalizations,suspected_icu,healthcare_worker_infections,healthcare_worker_deaths,source_url,date_time,new_confirmed_cases,new_deaths
274,2020-01-25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-25,0.0,0.0
273,2020-01-26,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cdph.ca.gov/Programs/OPA/Pages/NR2...,2020-01-26,2.0,0.0
272,2020-01-27,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-27,0.0,0.0
271,2020-01-28,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-28,0.0,0.0
270,2020-01-29,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-29,0.0,0.0
269,2020-01-30,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-30,0.0,0.0
268,2020-01-31,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cdph.ca.gov/Programs/OPA/Pages/NR2...,2020-01-31,1.0,0.0
267,2020-02-01,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-01,0.0,0.0
266,2020-02-02,6,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cdph.ca.gov/Programs/OPA/Pages/NR2...,2020-02-02,3.0,0.0
265,2020-02-03,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-03,0.0,0.0


In [17]:
x = np.array(data['date'],dtype=np.datetime64)
y = np.array(data['new_confirmed_cases'])

output_notebook() #or output_file()

p = figure(y_axis_label='New Confirmed Cases',
    title='new confirmed cases of Coronavirus',
    x_axis_type="datetime")

p.line(x, y)

show(p)

Loading BokehJS ...

In [18]:
data['date_time']=pd.to_datetime(data['date'])

output_notebook()

p = figure(y_axis_label='New Confirmed Cases',
    title='new confirmed cases of Coronavirus',
    x_axis_type="datetime")
r = p.line('date_time', 'new_confirmed_cases', source=data)
p.add_tools(HoverTool(
    tooltips=[
        ('date', '@date_time{%Y-%m-%d}'),
        ("new cases", "@new_confirmed_cases"),
    ],

    formatters={
        '@date_time': 'datetime',
    }
))

show(p)


Loading BokehJS ...

In [36]:
data['date_time']=pd.to_datetime(data['date'])
df=pd.DataFrame({'x':data['date_time'], 'y':data['new_confirmed_cases']})
source = ColumnDataSource(df)

output_notebook()

p = figure(y_axis_label='Number of Cases/Deaths',
    title='Coronavirus Tracker',
    x_axis_type="datetime")
r = p.line('x', 'y', source=source)
p.add_tools(HoverTool(
    tooltips=[
        ('date', '@x{%Y-%m-%d}'),
        ("new cases/deaths", "@y"),
    ],

    formatters={
        '@x': 'datetime',
    }
))




   

Loading BokehJS ...

In [39]:
from datetime import date, datetime

def update(sel):
    
    
    if sel=='Cases':
        r.data_source.data['y']=data['new_confirmed_cases']
    elif sel == 'Deaths':
        r.data_source.data['y']=data['new_deaths']
        
    dd = date.fromisoformat("2020-08-01")
    p.x_range.start = datetime(dd.year, dd.month, dd.day)
    push_notebook()

show(p, notebook_handle=True)

In [40]:
interact(update,sel=['Cases', 'Deaths'])

interactive(children=(Dropdown(description='sel', options=('Cases', 'Deaths'), value='Cases'), Output()), _dom…

<function __main__.update(sel)>